In [1]:
import pandas as pd
import numpy as np 

import torch
from torch import nn

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torch.utils.data import DataLoader,TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.metrics import *


In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

# device in which the model will be trained
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
df = pd.read_csv("../input/milk-dataset-test/test_data_df_galacticum_12.csv")
df = df.drop([df.columns[0], 'target'], axis=1)
df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_15591,feature_15592,feature_15593,feature_15594,feature_15595,feature_15596,feature_15597,feature_15598,feature_15599,substance
0,-220.4710,-145.0895,-110.0550,-90.95647,-78.79163,-70.40447,-64.83944,-60.07366,-56.79371,-53.84112,...,-125.4783,-124.0110,-122.8747,-121.1851,-120.0681,-118.2195,-117.2963,-116.0082,-114.7552,streptomycine
1,-241.0052,-162.3181,-124.1941,-103.45250,-89.78066,-81.02272,-74.25644,-69.16653,-65.26028,-62.09993,...,-257.2461,-256.5545,-256.1803,-255.7095,-255.3531,-254.5555,-253.6593,-253.3697,-252.4267,streptomycine
2,-769.0115,-682.1849,-629.4619,-592.12270,-564.07680,-542.61740,-525.04800,-510.96260,-499.05710,-488.86010,...,-477.7943,-478.2509,-478.8875,-479.3804,-479.9754,-479.9331,-480.9358,-481.3525,-481.4512,streptomycine
3,-382.0688,-284.5211,-233.6798,-202.84660,-182.32590,-167.33510,-156.45910,-147.82090,-140.76480,-134.94340,...,-388.3795,-385.1066,-381.6575,-377.9981,-373.9832,-370.1180,-365.8466,-361.4741,-357.3152,streptomycine
4,-388.5424,-297.8132,-249.1233,-218.21110,-197.36170,-182.45780,-170.66730,-161.70320,-154.37510,-148.37560,...,-498.0549,-497.4196,-497.2490,-497.0809,-496.3810,-496.1337,-495.7749,-495.3437,-495.1330,streptomycine


In [4]:
input_features = df.columns[:-1].tolist()
target = "substance"

In [5]:
# import some utilities to transform/preprocess our data:

from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Encoding categorical features
le = LabelEncoder()

In [6]:
X = df.iloc[:, :-1]
y = pd.DataFrame(le.fit_transform(df[target]), columns=[target])

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True)

In [8]:
y_train = np_utils.to_categorical(y_train, 3)
y_val = np_utils.to_categorical(y_val, 3)

In [9]:
train_tensor_dset = TensorDataset(
    torch.tensor(X_train.values, dtype=torch.float),
    torch.tensor(y_train, dtype=torch.float)
)

valid_tensor_dset = TensorDataset(
    torch.tensor(X_val.values, dtype=torch.float),
    torch.tensor(y_val, dtype=torch.float)
)

In [10]:
class SoftOrdering1DCNN(pl.LightningModule):

    def __init__(self, input_dim, output_dim, sign_size=64, cha_input=32, cha_hidden=512, 
                 K=5, dropout_input=0.2, dropout_hidden=0.2, dropout_output=0.2):
        super().__init__()

        hidden_size = sign_size*cha_input
        sign_size1 = sign_size
        sign_size2 = sign_size//2
        output_size = (sign_size//4) * cha_hidden

        self.hidden_size = hidden_size
        self.cha_input = cha_input
        self.cha_hidden = cha_hidden
        self.K = K
        self.sign_size1 = sign_size1
        self.sign_size2 = sign_size2
        self.output_size = output_size
        self.dropout_input = dropout_input
        self.dropout_hidden = dropout_hidden
        self.dropout_output = dropout_output

        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout1 = nn.Dropout(dropout_input)
        dense1 = nn.Linear(input_dim, hidden_size, bias=False)
        self.dense1 = nn.utils.weight_norm(dense1)

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_input)
        conv1 = conv1 = nn.Conv1d(
            cha_input, 
            cha_input*K, 
            kernel_size=5, 
            stride = 1, 
            padding=2,  
            groups=cha_input, 
            bias=False)
        self.conv1 = nn.utils.weight_norm(conv1, dim=None)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_input*K)
        self.dropout_c2 = nn.Dropout(dropout_hidden)
        conv2 = nn.Conv1d(
            cha_input*K, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv2 = nn.utils.weight_norm(conv2, dim=None)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_hidden)
        self.dropout_c3 = nn.Dropout(dropout_hidden)
        conv3 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False)
        self.conv3 = nn.utils.weight_norm(conv3, dim=None)
        

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_hidden)
        conv4 = nn.Conv1d(
            cha_hidden, 
            cha_hidden, 
            kernel_size=5, 
            stride=1, 
            padding=2, 
            groups=cha_hidden, 
            bias=False)
        self.conv4 = nn.utils.weight_norm(conv4, dim=None)

        self.avg_po_c4 = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm2 = nn.BatchNorm1d(output_size)
        self.dropout2 = nn.Dropout(dropout_output)
        dense2 = nn.Linear(output_size, output_dim, bias=False)
        self.dense2 = nn.utils.weight_norm(dense2)

        self.loss = nn.MSELoss()

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = nn.functional.celu(self.dense1(x))

        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1)

        x = self.batch_norm_c1(x)
        x = nn.functional.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = nn.functional.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = nn.functional.relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.conv4(x)
        x =  x + x_s
        x = nn.functional.relu(x)

        x = self.avg_po_c4(x)

        x = self.flt(x)

        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.dense2(x)

        return x

    def training_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('train_loss', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        X, y = batch
        y_hat = self.forward(X)
        loss = self.loss(y_hat, y)
        self.log('valid_loss', loss)
        
    def test_step(self, batch, batch_idx):
        X, y = batch
        y_logit = self.forward(X)
        y_probs = y_logit.argmax(1).cpu().numpy()
        loss = self.loss(y_logit, y)
        metric1 = accuracy_score(y.argmax(1).cpu().numpy(), y_probs)
        metric2 = precision_score(y.argmax(1).cpu().numpy(), y_probs, average='macro')
        metric3 = recall_score(y.argmax(1).cpu().numpy(), y_probs, average='macro')
        self.log('test_accuracy', metric1)
        self.log('test_precision', metric2)
        self.log('test_recall', metric3)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        scheduler = {
            'scheduler': ReduceLROnPlateau(
                optimizer, 
                mode="min", 
                factor=0.5, 
                patience=5, 
                min_lr=1e-6),
            'interval': 'epoch',
            'frequency': 1,
            'reduce_on_plateau': True,
            'monitor': 'valid_loss',
        }
        return [optimizer], [scheduler]

In [14]:
model = SoftOrdering1DCNN(
    input_dim=len(input_features), 
    output_dim=3, 
)

early_stop_callback = EarlyStopping(
   monitor='valid_loss',
   min_delta=.0,
   patience=30,
   verbose=True,
   mode='min'a
)

trainer = pl.Trainer(callbacks=[early_stop_callback], min_epochs=10, max_epochs=200, gpus=1)

In [16]:
trainer.fit(
    model, 
    DataLoader(train_tensor_dset, batch_size=8, shuffle=True, num_workers=4),
    DataLoader(valid_tensor_dset, batch_size=1024, shuffle=False, num_workers=4)
)

Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1896: PossibleUserWarning: The number of training batches (3) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 4it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [17]:
trainer.test(model, DataLoader(valid_tensor_dset, batch_size=2048, shuffle=False, num_workers=4))

Testing: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.6666666666666666     │
│      test_precision       │    0.3333333333333333     │
│        test_recall        │    0.26666666666666666    │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.6666666666666666,
  'test_precision': 0.3333333333333333,
  'test_recall': 0.26666666666666666}]

In [18]:
np.unique(df[target])

array(['milk', 'streptomycine', 'tetracycline'], dtype=object)

In [19]:
test = pd.read_csv("../input/milk-dataset-test/test_data_df_galacticum_12.csv")
test = test.drop([test.columns[0], 'target'], axis=1)
test.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_15591,feature_15592,feature_15593,feature_15594,feature_15595,feature_15596,feature_15597,feature_15598,feature_15599,substance
0,-220.4710,-145.0895,-110.0550,-90.95647,-78.79163,-70.40447,-64.83944,-60.07366,-56.79371,-53.84112,...,-125.4783,-124.0110,-122.8747,-121.1851,-120.0681,-118.2195,-117.2963,-116.0082,-114.7552,streptomycine
1,-241.0052,-162.3181,-124.1941,-103.45250,-89.78066,-81.02272,-74.25644,-69.16653,-65.26028,-62.09993,...,-257.2461,-256.5545,-256.1803,-255.7095,-255.3531,-254.5555,-253.6593,-253.3697,-252.4267,streptomycine
2,-769.0115,-682.1849,-629.4619,-592.12270,-564.07680,-542.61740,-525.04800,-510.96260,-499.05710,-488.86010,...,-477.7943,-478.2509,-478.8875,-479.3804,-479.9754,-479.9331,-480.9358,-481.3525,-481.4512,streptomycine
3,-382.0688,-284.5211,-233.6798,-202.84660,-182.32590,-167.33510,-156.45910,-147.82090,-140.76480,-134.94340,...,-388.3795,-385.1066,-381.6575,-377.9981,-373.9832,-370.1180,-365.8466,-361.4741,-357.3152,streptomycine
4,-388.5424,-297.8132,-249.1233,-218.21110,-197.36170,-182.45780,-170.66730,-161.70320,-154.37510,-148.37560,...,-498.0549,-497.4196,-497.2490,-497.0809,-496.3810,-496.1337,-495.7749,-495.3437,-495.1330,streptomycine


In [20]:
test.loc[test.substance == 'cephazoline', 'substance'] = 'cefazolin'

In [21]:
X_test = test.iloc[:, :-1]
y_test = pd.DataFrame(le.transform(test['substance']), columns=['substance'])

In [22]:
y_test = np_utils.to_categorical(y_test, 3)

In [23]:
test_tensor_dset = TensorDataset(
    torch.tensor(X_test.values, dtype=torch.float),
    torch.tensor(y_test, dtype=torch.float)
)

In [24]:
trainer.test(model, DataLoader(test_tensor_dset, batch_size=2048
                               , shuffle=False, num_workers=4))

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.8148148148148148     │
│      test_precision       │    0.8296296296296296     │
│        test_recall        │    0.8078431372549019     │
└───────────────────────────┴───────────────────────────┘

[{'test_accuracy': 0.8148148148148148,
  'test_precision': 0.8296296296296296,
  'test_recall': 0.8078431372549019}]

In [25]:
torch.save(model.state_dict(), "galacticum_cnn_milk_v1.pth")

In [1]:
# Loading model 

In [ ]:
model = SoftOrdering1DCNN(
    input_dim=len(input_features), 
    output_dim=3, 
)
model.load_state_dict(torch.load(PATH))